# 调整benchmark.json以使其合法

In [10]:
import json

OUTPUT_DIR = "results"

def load_existing_results():
    evaluated_key = dict()
    # 读取已有的结果
    with open(f"{OUTPUT_DIR}/benchmark.json", "r") as f:
        content = json.load(f)
    correct = []
    for idx, eval_res in enumerate(content):
        if eval_res is None:
            continue
        eval_setting = {
            "task_name": eval_res["task_name"],
            "model_name": eval_res["model_name"],
            "chunking_strategy": eval_res["chunking_strategy"],
            "chunk_size": eval_res["chunk_size"],
        }
        key = json.dumps(eval_setting, sort_keys=True)
        if key in evaluated_key:
            print(f"key: {key}")
            print(f"found@{idx}: {eval_res}")
            print(f"duplicate@{evaluated_key[key][1]}: {evaluated_key[key][0]}")
            print("\n\n")
            continue
        correct.append(eval_res)
        evaluated_key[key] = (eval_res, idx)
    with open(f"{OUTPUT_DIR}/benchmark.json", "w", encoding="utf-8") as f:
        json.dump(correct, f, ensure_ascii=False, indent=4)

load_existing_results()

key: {"chunk_size": 1024, "chunking_strategy": "fixed_token", "model_name": "BAAI/bge-m3", "task_name": "CrosslingualRetrievalPaperEn2ZhChunked"}
found@291: {'dataset_revision': '', 'task_name': 'CrosslingualRetrievalPaperEn2ZhChunked', 'mteb_version': '1.14.20', 'scores': {'dev': [{'ndcg_at_1': 0.90675, 'ndcg_at_3': 0.93728, 'ndcg_at_5': 0.94154, 'ndcg_at_10': 0.94467, 'ndcg_at_20': 0.9461, 'map_at_1': 0.90671, 'map_at_3': 0.93009, 'map_at_5': 0.93245, 'map_at_10': 0.93375, 'map_at_20': 0.93416, 'recall_at_1': 0.90671, 'recall_at_3': 0.95792, 'recall_at_5': 0.96825, 'recall_at_10': 0.97789, 'recall_at_20': 0.9835, 'precision_at_1': 0.90675, 'precision_at_3': 0.31932, 'precision_at_5': 0.19366, 'precision_at_10': 0.0978, 'precision_at_20': 0.04918, 'mrr_at_1': 0.9067509796936231, 'mrr_at_3': 0.930130032062699, 'mrr_at_5': 0.9324946562166017, 'mrr_at_10': 0.9337893397175993, 'mrr_at_20': 0.9341936045850086, 'main_score': 0.94467, 'hf_subset': 'default', 'languages': ['en-zh']}]}, 'evalu

# 统计各数据集的大小

In [3]:
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

from chunked_pooling.chunked_eval_tasks import *
from chunked_pooling.wrappers import load_model
from transformers import AutoModel, AutoTokenizer

task_name_to_cls = get_eval_tasks()
model_name = "jinaai/jina-embeddings-v2-small-en"
model, has_instructions = load_model(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model.eval()

sample_count = dict()
for task_name, task_cls in task_name_to_cls.items():
    chunking_args = {
        "chunk_size": 1024,
        "n_sentences": 5,
        "chunking_strategy": "fixed_text",
        "model_has_instructions": has_instructions,
        "embedding_model_name": model_name,
    }

    task = task_cls(
        tokenizer=tokenizer,
        prune_size=None,
        truncate_max_length=False,
        **chunking_args,
    )

    task.load_data()
    if "test" in task.queries:
        queries = task.queries["test"]
        corpus = task.corpus["test"]
    elif "dev" in task.queries:
        queries = task.queries["dev"]
        corpus = task.corpus["dev"]
    else:
        continue
    
    num_queries = len(queries)
    num_corpus = len(corpus)
    sample_count[task_name] = {"queries": num_queries, "corpus": num_corpus}
    print(f"{task_name}: NumOfQueries: {num_queries}, NumOfCorpus:{num_corpus}, total: {num_queries+num_corpus}")

    with open("sample_count.json", "w") as f:
        json.dump(sample_count, f)

Map: 100%|██████████| 1406/1406 [00:00<00:00, 23623.07 examples/s]


ArguAnaChunked: NumOfQueries: 1406, NumOfCorpus:8674, total: 10080


Map: 100%|██████████| 4681/4681 [00:00<00:00, 30193.97 examples/s]


ClimateFEVERChunked: NumOfQueries: 1535, NumOfCorpus:5416593, total: 5418128


Map: 100%|██████████| 43515/43515 [00:01<00:00, 32656.56 examples/s]


...

CrosslingualRetrievalPaperZh2EnChunked: NumOfQueries: 22515, NumOfCorpus:5101, total: 27616


Using the latest cached version of the dataset since maidalun1020/CrosslingualRetrievalWikiEn2Zh couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/zyh/.cache/huggingface/datasets/maidalun1020___crosslingual_retrieval_wiki_en2_zh/default/0.0.0/e005f084db042b883a672a62f111c15c2e570dee (last modified on Wed Oct  9 15:57:00 2024).
Using the latest cached version of the dataset since maidalun1020/CrosslingualRetrievalWikiEn2Zh-qrels couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/zyh/.cache/huggingface/datasets/maidalun1020___crosslingual_retrieval_wiki_en2_zh-qrels/default/0.0.0/48ffbd2c616e176d786e6a1b42c3d6af79d4964b (last modified on Wed Oct  9 16:01:12 2024).


CrosslingualRetrievalWikiEn2ZhChunked: NumOfQueries: 34060, NumOfCorpus:6506, total: 40566


Using the latest cached version of the dataset since maidalun1020/CrosslingualRetrievalWikiZh2En couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/zyh/.cache/huggingface/datasets/maidalun1020___crosslingual_retrieval_wiki_zh2_en/default/0.0.0/8d0fceb8bda8345eea22c42f4b57b2ceebd51010 (last modified on Wed Oct  9 15:59:42 2024).
Using the latest cached version of the dataset since maidalun1020/CrosslingualRetrievalWikiZh2En-qrels couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/zyh/.cache/huggingface/datasets/maidalun1020___crosslingual_retrieval_wiki_zh2_en-qrels/default/0.0.0/97e68a0530c3bb26403fe76837e5df131bad96f1 (last modified on Wed Oct  9 16:01:40 2024).


CrosslingualRetrievalWikiZh2EnChunked: NumOfQueries: 34062, NumOfCorpus:6504, total: 40566


In [4]:
lines = []
with open("master.log", "r") as f:
    lines = f.readlines()

dedup = set()
for line in lines:
    if line in dedup:
        print("Duplicate: " + line)
    dedup.add(line.strip())